# Análisis exploratorio de Datos GEDI

Análisis exploratorio de datos estructurales de vegetación provenientes de pulsor de Láser del proyecto GEDI. Se entrena con modelos de Palsar-2, Sentinel 1 y 2. Se usa métricas como el estado de la vegtación y estructura de datos Radar.

## 1.Importar librerías

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

## 2.Cargar Datos

### Datos CSV

Se carga datos de sentinel 1 y 2, provenientes de un archivo .csv. Además, se agrega los datos palsar que está en una base de datos .db, y se adjuntar por el "id".

In [2]:
datos_sent_1_2 = pd.read_csv('DATA\DATA_GEDI_ESTRCTURA_S1_S2.csv')
datos_sent_1_2.head()

,RFDI,id,DPDD,DpRVIVV,VH,VV,B11,B11_contrast,B11_corr,B11_diss,...,SAVI,Slope,elevation,fhd_normal,N_efectivo_estratos,month,time,year,lon,lat
0,0.147805,37,0.181490,0.686560,0.044054,0.212611,1402.5,9.680916e+11,0.584645,621104.545455,...,0.505120,0.927586,369,3.106986,8.615807,8,2019-08-01 08:00:00,2019,-77.476179,-1.117864
1,0.198458,38,0.089690,0.905287,0.028707,0.098134,1550.0,8.835777e+11,0.534480,610385.227273,...,0.551084,1.311566,369,3.319075,9.980243,8,2019-08-01 08:00:00,2019,-77.476179,-1.118134
2,-0.048516,39,0.229352,0.503965,0.040866,0.283487,1488.5,9.679577e+11,0.702365,645921.590909,...,0.528486,7.435666,371,3.352817,10.216414,8,2019-08-01 08:00:00,2019,-77.475910,-1.118673
3,0.130420,40,0.145603,0.909069,0.046798,0.159116,1104.5,7.593858e+11,0.585595,524107.954545,...,0.407068,3.340522,373,3.316862,9.964946,8,2019-08-01 08:00:00,2019,-77.475371,-1.118942
4,0.269977,41,0.228092,0.577314,0.046556,0.276014,1411.0,6.108494e+11,0.610041,456927.272727,...,0.529927,1.311566,372,3.193762,9.149938,8,2019-08-01 08:00:00,2019,-77.475101,-1.119481


### Datos .DB

In [3]:
conn = sqlite3.connect('DATA/palsar_data.db')
df_paslar_2 = pd.read_sql("SELECT * FROM palsar_data", conn)
df_paslar_2.head()

,id,DpRVIHH,HH,HV,HV_contrast,HV_corr,HV_diss,HV_ent,HV_var
0,37,1.704391,4888.5,3629.5,2.891250e+07,0.211371,4273.364773,5.345972,1.827252e+07
1,38,1.603085,5595.0,3742.0,2.763441e+07,0.282679,4200.443182,5.345972,1.928422e+07
2,39,2.097031,3942.0,4344.0,3.168169e+07,0.263496,4436.804545,5.345972,2.145946e+07
3,40,1.739160,4934.0,3795.5,3.218097e+07,0.304248,4364.476136,5.345972,2.320166e+07
4,41,1.460045,5038.0,2896.0,3.747875e+07,0.350021,4808.880682,5.345972,2.874796e+07


### Unión de datos

In [4]:
df_union_gedi = pd.merge(datos_sent_1_2, df_paslar_2, how='inner', left_on='id', right_on='id')
df_union_gedi.head()

,RFDI,id,DPDD,DpRVIVV,VH,VV,B11,B11_contrast,B11_corr,B11_diss,...,lon,lat,DpRVIHH,HH,HV,HV_contrast,HV_corr,HV_diss,HV_ent,HV_var
0,0.147805,37,0.181490,0.686560,0.044054,0.212611,1402.5,9.680916e+11,0.584645,621104.545455,...,-77.476179,-1.117864,1.704391,4888.5,3629.5,2.891250e+07,0.211371,4273.364773,5.345972,1.827252e+07
1,0.198458,38,0.089690,0.905287,0.028707,0.098134,1550.0,8.835777e+11,0.534480,610385.227273,...,-77.476179,-1.118134,1.603085,5595.0,3742.0,2.763441e+07,0.282679,4200.443182,5.345972,1.928422e+07
2,-0.048516,39,0.229352,0.503965,0.040866,0.283487,1488.5,9.679577e+11,0.702365,645921.590909,...,-77.475910,-1.118673,2.097031,3942.0,4344.0,3.168169e+07,0.263496,4436.804545,5.345972,2.145946e+07
3,0.130420,40,0.145603,0.909069,0.046798,0.159116,1104.5,7.593858e+11,0.585595,524107.954545,...,-77.475371,-1.118942,1.739160,4934.0,3795.5,3.218097e+07,0.304248,4364.476136,5.345972,2.320166e+07
4,0.269977,41,0.228092,0.577314,0.046556,0.276014,1411.0,6.108494e+11,0.610041,456927.272727,...,-77.475101,-1.119481,1.460045,5038.0,2896.0,3.747875e+07,0.350021,4808.880682,5.345972,2.874796e+07


### Exportar unión de datos

In [5]:
df_union_gedi.to_csv('DATA/df_union_gedi.csv', index=False)